In [60]:
from pandas import read_stata
import pandas as pd
from lcsscaseapi.client import LCSSClient
from lcsscaseapi.types import USCircuitCaseMeta
import datetime
import os
from dotenv import load_dotenv

In [4]:
df = read_stata("BloombergCASELEVEL_Touse.dta")

print(len(df["caseid"]))

387898


In [56]:
def create_case_meta(row):
    case_id = row['caseid']
    circuit_num = row['Circuit']
    circuit_name = USCircuitCaseMeta.CIRCUITS[circuit_num]
    self_cite = row['citation']
    docket_number = row['docketnumber']
    if not pd.isnull(row['date']):
        date = datetime.date(int(row['year']), int(row['month']), int(row['day']))
    else:
        date = None
    tags = construct_tags(row)
    outcome = construct_outcome(row)

    return USCircuitCaseMeta(case_id=case_id, circuit_name=circuit_name, self_cite=self_cite, 
                                docket_number=docket_number, date=date, tags=tags, outcome = outcome)

def construct_tags(row):
    tags = []
    if row['Criminal'] == 1:
        tags.append('CRIMINAL')
    
    if row['Civil_Rights'] == 1:
        tags.append('CIVIL RIGHTS')

    if row['First_Amendment'] == 1:
        tags.append('FIRST AMENDMENT')
    
    if row['Due_Process'] == 1:
        tags.append('DUE PROCESS')
    
    if row['Privacy'] == 1:
        tags.append('PRIVACY')
    
    if row['Labor_Relations'] == 1:
        tags.append('LABOR RELATIONS')
    
    if row['Econ_Activity'] == 1:
        tags.append('ECONOMIC ACTIVITY')

    if row['Miscellanous'] == 1:
        tags.append('MISCELLANEOUS')

    return tags

def construct_outcome(row):
    outcomes = []

    if row["Affirmed"] == 1:
        outcomes.append("AFFIRMED")
    
    if row["AffirmedInPart"] == 1:
        outcomes.append("AFFIRMED (IN PART)")
    
    if row["Reversed"] == 1:
        outcomes.append("REVERSED")

    if row["ReversedInPart"] == 1:
        outcomes.append("REVERSED (IN PART)")
    
    if row["Vacated"] == 1:
        outcomes.append("VACATED")

    if row["VacatedInPart"] == 1:
        outcomes.append("VACATED (IN PART)")

    if row["Remanded"] == 1:
        outcomes.append("REMANDED")

    if len(outcomes) == 0:
        return None
    else:
        return ",".join(outcomes)



    

In [57]:
# takes 39.5s to run
cases = df.apply(create_case_meta, axis=1)

print(len(cases))

387898


In [64]:
load_dotenv()

USERNAME = os.getenv('ACCOUNT')
PWD = os.getenv('PASSWORD')

client = LCSSClient(username=USERNAME, password=PWD)

sudolcss
affirmedinpart


In [ ]:
client.upload_us_cases(cases)